# [Flower](https://github.com/adap/flower) client with CIFAR-10 dataset


In [ ]:
import os
import pathlib

import flwr as fl
import tensorflow as tf
import numpy as np

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

INPUTS_PATH = os.environ.get("NEVERMINED_INPUTS_PATH")

In [ ]:
# Read input files
def load_data():
    x_train = None
    y_train = None
    x_test = None
    y_test = None
    for f in pathlib.Path(INPUTS_PATH).rglob("*/*.npy"):
        if f.name.startswith("x_train"):
            x_train = np.load(f.as_posix())
        elif f.name.startswith("y_train"):
            y_train = np.load(f.as_posix())
        elif f.name.startswith("x_test"):
            x_test = np.load(f.as_posix())
        elif f.name.startswith("y_test"):
            y_test = np.load(f.as_posix())

    return (x_train, y_train), (x_test, y_test)


In [ ]:
# Load and compile Keras model
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = load_data()

# Define Flower client
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self):  # type: ignore
        return model.get_weights()

    def fit(self, parameters, config):  # type: ignore
        model.set_weights(parameters)
        model.fit(x_train, y_train, epochs=1, batch_size=32)
        return model.get_weights(), len(x_train), {}

    def evaluate(self, parameters, config):  # type: ignore
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(x_test, y_test)
        return loss, len(x_test), {"accuracy": accuracy}

In [ ]:
# Start Flower client
fl.client.start_numpy_client("172.17.0.2:8080", client=CifarClient())